In [1]:
from deeploy import Client, DeployOptions 

## Initialise Deeploy

In [2]:
client_options = {
    'access_key': 'AKIAYTKJI1RK3TYI1XUG',
    'secret_key': 'l2qBUUPg3Tb2fiNLhoeaNbb26AbHFqooB3o07Avz',
    'host': 'ute.deeploy.ml',
    'workspace_id': 'ac8c6126-6782-11ea-bc55-0242ac130003',
    'local_repository_path': '/Users/larssuanet/Documents/deeploy/sample_models/deeploy-client-test',
}
client = Client(**client_options)

## Train model

In [3]:
import sklearn
import shap

X,y = shap.datasets.adult()
X_display,y_display = shap.datasets.adult(display=True)
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=7)

knn = sklearn.neighbors.KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

## Create explainer

In [4]:
f = lambda x: knn.predict_proba(x)[:,1]
med = X_train.median().values.reshape((1,X_train.shape[1]))
explainer = shap.KernelExplainer(f, med)

## Deploy model and explainer to Deeploy

In [5]:
deploy_options = {
    'model': knn, 
    'options': DeployOptions(**{
        'name': 'client test',
        'model_serverless': False,
        'explainer_serverless': False,
        'description': 'the first test',
    }), 
    'explainer': explainer,
}
client.deploy(**deploy_options)

Deployment(commit_id='11861532-1ae1-4d99-941a-de2837cc3bfe', name='client test', workspace_id='ac8c6126-6782-11ea-bc55-0242ac130003', description='the first test', example_input=None, example_output=None, model_type=<ModelType.SKLEARN: 2>, model_class_name=None, method=<PredictionMethod.PREDICT: 'predict'>, model_serverless=False, explainer_type=<ExplainerType.SHAP_KERNEL: 4>, explainer_serverless=False, status=4, is_archived=False, s3_link='ac8c6126-6782-11ea-bc55-0242ac130003/repositories/9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1/commits/11861532-1ae1-4d99-941a-de2837cc3bfe', owner_id='a17c0049-043e-4730-aa5c-30c75a1914fc', repository_id='9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1', kf_serving_id=None, public_url=None, id='c122b939-da68-431d-8eec-4797271243b4', created_at='2021-02-17T15:51:34.427Z', updated_at='2021-02-17T15:51:34.427Z')

## XGBoost


In [3]:
import xgboost as xgb
from sklearn.datasets import load_iris

iris = load_iris()
y = iris['target']
X = iris['data']
dtrain = xgb.DMatrix(X, label=y)
param = {'max_depth': 6,
            'eta': 0.1,
            'silent': 1,
            'nthread': 4,
            'num_class': 10,
            'objective': 'multi:softmax'
            }
xgb_model = xgb.train(params=param, dtrain=dtrain)

deploy_options = {
    'model': xgb_model, 
    'options': DeployOptions(**{
        'name': 'client test xgboost',
        'model_serverless': False,
        'explainer_serverless': False,
        'description': 'the first test',
    }),
    'overwrite': True,
}
client.deploy(**deploy_options)

Deployment(commit_id='cc55b79b-1282-4b06-a20e-68601d11d341', name='client test xgboost', workspace_id='ac8c6126-6782-11ea-bc55-0242ac130003', description='the first test', example_input=None, example_output=None, model_type=<ModelType.XGBOOST: 3>, model_class_name=None, method=<PredictionMethod.PREDICT: 'predict'>, model_serverless=False, explainer_type=<ExplainerType.NO_EXPLAINER: 0>, explainer_serverless=False, status=4, is_archived=False, s3_link='ac8c6126-6782-11ea-bc55-0242ac130003/repositories/9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1/commits/cc55b79b-1282-4b06-a20e-68601d11d341', owner_id='a17c0049-043e-4730-aa5c-30c75a1914fc', repository_id='9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1', kf_serving_id=None, public_url=None, id='bd3361a8-c7f2-4ff4-84be-47edad7390f6', created_at='2021-02-18T10:58:07.756Z', updated_at='2021-02-18T10:58:07.756Z')

## Pytorch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                            shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                            shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

            # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
deploy_options = {
    'model': net, 
    'options': DeployOptions(**{
        'name': 'client test pytorch',
        'model_serverless': False,
        'explainer_serverless': False,
        'description': 'the first test',
        'model_class_name': 'Net',
        'pytorch_model_file_path': '/Users/larssuanet/Documents/deeploy/deeploy-python-client/model.py',
    }),
    'overwrite': True,
}
client.deploy(**deploy_options)

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.219
[1,  4000] loss: 1.860
[1,  6000] loss: 1.681
[1,  8000] loss: 1.580
[1, 10000] loss: 1.510
[1, 12000] loss: 1.474
[2,  2000] loss: 1.382
[2,  4000] loss: 1.347
[2,  6000] loss: 1.317
[2,  8000] loss: 1.315
[2, 10000] loss: 1.283
[2, 12000] loss: 1.260


KeyError: 'data'

In [6]:
import inspect

base_classes = list(map(lambda x: x.__module__ + '.' +
                                x.__name__, inspect.getmro(type(net))))
print(base_classes)

['__main__.Net', 'torch.nn.modules.module.Module', 'builtins.object']


In [10]:
import inspect
import sys

print(inspect.getsource(inspect.getmodule(inspect.currentframe())))

TypeError: module, class, method, function, traceback, frame, or code object was expected, got NoneType

## TensorFlow & Alibi AnchorImage

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np
# from alibi.explainers import AnchorImage

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = np.reshape(x_train, x_train.shape + (1,))
x_test = np.reshape(x_test, x_test.shape + (1,))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

def model():
    x_in = Input(shape=(28, 28, 1))
    x = Conv2D(filters=64, kernel_size=2, padding='same', activation='relu')(x_in)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(filters=32, kernel_size=2, padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(10, activation='softmax')(x)
    
    cnn = Model(inputs=x_in, outputs=x_out)
    cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return cnn

cnn = model()
cnn.fit(x_train, y_train, batch_size=64, epochs=3)

# def superpixel(image, size=(4, 7)):
#     segments = np.zeros([image.shape[0], image.shape[1]])
#     row_idx, col_idx = np.where(segments == 0)
#     for i, j in zip(row_idx, col_idx):
#         segments[i, j] = int((image.shape[1]/size[1]) * (i//size[0]) + j//size[1])
#     return segments

# predict_fn = lambda x: cnn.predict(x)
# image_shape = x_train[idx].shape

# explainer = AnchorImage(predict_fn, image_shape, segmentation_fn=superpixel)

deploy_options = {
    'model': cnn, 
    'options': DeployOptions(**{
        'name': 'client test tensorflow alibi',
        'model_serverless': False,
        'explainer_serverless': False,
        'description': 'the first test',
    }),
    'overwrite': True,
    # 'explainer': explainer,
}
client.deploy(**deploy_options)

Epoch 1/3
938/938 [==============================] - 40s 42ms/step - loss: 0.8036 - accuracy: 0.7012
Epoch 2/3
938/938 [==============================] - 40s 43ms/step - loss: 0.4309 - accuracy: 0.8418
Epoch 3/3
938/938 [==============================] - 38s 41ms/step - loss: 0.3767 - accuracy: 0.8649
INFO:tensorflow:Assets written to: /Users/larssuanet/Documents/deeploy/sample_models/deeploy-client-test/model/1/assets
INFO:tensorflow:Assets written to: /Users/larssuanet/Documents/deeploy/sample_models/deeploy-client-test/model/1/assets


Deployment(commit_id='976b4f7d-0ebd-44ce-999c-b1e2b2b32343', name='client test tensorflow alibi', workspace_id='ac8c6126-6782-11ea-bc55-0242ac130003', description='the first test', example_input=None, example_output=None, model_type=<ModelType.TENSORFLOW: 0>, model_class_name=None, method=<PredictionMethod.PREDICT: 'predict'>, model_serverless=False, explainer_type=<ExplainerType.NO_EXPLAINER: 0>, explainer_serverless=False, status=4, is_archived=False, s3_link='ac8c6126-6782-11ea-bc55-0242ac130003/repositories/9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1/commits/976b4f7d-0ebd-44ce-999c-b1e2b2b32343', owner_id='a17c0049-043e-4730-aa5c-30c75a1914fc', repository_id='9b4f2a96-0da0-4d2c-8b14-595c2ddcecc1', kf_serving_id=None, public_url=None, id='68360aaa-5994-433f-be04-488a68b70aa0', created_at='2021-02-19T12:57:34.932Z', updated_at='2021-02-19T12:57:34.932Z')

In [8]:
import inspect

base_classes = list(map(lambda x: x.__module__ + '.' +
                                x.__name__, inspect.getmro(type(cnn))))
print(base_classes)

['tensorflow.python.keras.engine.functional.Functional', 'tensorflow.python.keras.engine.training.Model', 'tensorflow.python.keras.engine.base_layer.Layer', 'tensorflow.python.module.module.Module', 'tensorflow.python.training.tracking.tracking.AutoTrackable', 'tensorflow.python.training.tracking.base.Trackable', 'tensorflow.python.keras.utils.version_utils.LayerVersionSelector', 'tensorflow.python.keras.utils.version_utils.ModelVersionSelector', 'builtins.object']


NameError: name '__dir__' is not defined